# RLとFenchel Rockafellar双対性とDICE

参考
* [Reinforcement Learning via Fenchel-Rockafellar Duality](https://arxiv.org/abs/2001.01866)
* [Dual DICE](https://arxiv.org/abs/1906.04733)
* [AlgaeDICE: Policy Gradient from Arbitrary Experience](https://arxiv.org/abs/1912.02074)
* [GenDICE: Generalized Offline Estimation of Stationary Values](https://arxiv.org/abs/2002.09072)

これもわかりやすいかも
* [A Note on Optimization Formulations of Markov Decision Processes](https://arxiv.org/abs/2012.09417)

凸共役を使ったRLの問題の変形について見ていきます．


## Fenchel共役について

凸関数$f : \Omega \to \mathbb{R}$について，そのFenchel共役$f_*$は

$$
f_*(y):=\max _{x \in \Omega}\langle x, y\rangle-f(x),
$$

で表されます．また，

* $f$は$\{x \in \Omega: f(x)<\infty\}$が空ではなく，$f(x)>-\infty$ for all $x \in \Omega$であるときに，properであるといいます．
* $f$は$\{x \in \Omega: f(x)>\alpha\}$が$\alpha \in \mathbb{R}$について開集合であるときにlower semi-continuousであるといいます([これ](https://math.libretexts.org/Bookshelves/Analysis/Introduction_to_Mathematical_Analysis_I_(Lafferriere_Lafferriere_and_Nguyen)/03%3A_Limits_and_Continuity/3.07%3A_Lower_Semicontinuity_and_Upper_Semicontinuity)参考)．

$f$がproper，凸，そしてlower semi-continuousな関数であれば，$f_*$もproper，凸，そしてlower semi-continuousになります．

**f-divergenceの凸共役**

次の凸関数$f$のダイバージェンスを考えましょう．
$$
D_f(x \| p)=\mathbb{E}_{z \sim p}\left[f\left(\frac{x(z)}{p(z)}\right)\right]
$$

このとき，いくつかの仮定のもとで，$f$ダイバージェンスの凸共役は

$$
\begin{aligned}
g(y) & =\max _{x: \mathcal{Z} \rightarrow \mathbb{R}} \sum_z x(z) \cdot y(z)-\mathbb{E}_{z \sim p}[f(x(z) / p(z))] \\
& =\mathbb{E}_{z \sim p}\left[\max _{x(z) \in \mathbb{R}} x(z) \cdot y(z) / p(z)-f(x(z) / p(z))\right] \\
& =\mathbb{E}_{z \sim p}\left[f_*(y(z))\right] .
\end{aligned}
$$

になります．

例えばKLダイバージェンスなら，$g(y)=\log \mathbb{E}_{z \sim p}[\exp y(z)]$になります．

---

**Fenchel Rockafellar Duality**

次の主問題を考えます．

$$
\min _{x \in \Omega} J_{\mathrm{P}}(x):=f(x)+g(A x),
$$

ここで，$f, g$は凸，lower semi-continuousな関数です．また，$A$は行列です．
この双対問題は

$$
\max _{y \in \Omega^*} J_{\mathrm{D}}:=-f_*\left(-A_* y\right)-g_*(y)
$$

で与えられます．ここで$\langle y, A x\rangle=\left\langle A_* y, x\right\rangle$です．


---

**Lagrangian**

次の制約付き最適化問題を考えましょう．

$$
\min _x f(x) \text { s.t. } A x \geq b
$$

これを$\min _x f(x)+g(x) \text { for } g(x)=\delta_{\mathbb{R}_{-}}(-A x+b)$
を使って表した上でFenchel双対を取れば，

$$
\max _y\langle y, b\rangle-f_*\left(A_* y\right) \quad \text { s.t. } \quad y \geq 0
$$

です．さらに凸共役の定義にもどれば，
$$
\min _x \max _{y \geq 0}\langle y, b\rangle-\left\langle x, A_* y\right\rangle+f(x)
$$
と同じです．$A^*$の定義を使えば

$$
\min _x \max _{y \geq 0} \underbrace{\langle y, b-A x\rangle+f(x)}_{L(x, y)} .
$$
と同じです．


## Lagrangianを使った方策評価

線型計画法を使うと，方策評価は次の最適化で解けます．

$$
\begin{aligned}
\rho(\pi)=\min _Q(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right] \\
\text { s.t. } Q(s, a) \geq R(s, a)+\gamma \cdot \mathcal{P}^\pi Q(s, a), \\
\forall(s, a) \in S \times A .
\end{aligned}
$$

これをLagrangianで変形すると，

$$
\rho(\pi)=\min _Q \max _{d \geq 0}(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) 
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+ 
\sum_{s, a} d(s, a) \cdot\left(R(s, a)+\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right) .
$$

によって方策評価ができます．ここで，$d$の解は$d^\pi$です．制約の部分では$\sum_{s, a}$をとっていますが，これは一般に無限なことが多く，和を取るのが難しいです．
オフラインの設定で分布$d^\mathcal{D}$にアクセスできる場合，$\zeta(s, a)=\frac{d(s, a)}{d^{\mathcal{D}}(s, a)}$を使ったimportance samplingを使えば和を取るのが回避できます．

実際，

$$
\begin{aligned}
& \min _Q \max _{\zeta \geq 0} L(Q, \zeta) \\
& :=(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[\zeta(s, a) \cdot\left(R(s, a)+\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)\right]\\
& =(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{\substack{\left(s, a, s^{\prime}\right) \sim d^{\mathcal{D}} \\ a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[\zeta(s, a) \cdot\left(R(s, a)+\gamma Q\left(s^{\prime}, a^{\prime}\right)-Q(s, a)\right)\right]
\end{aligned}
$$

の形に変形でき，これはデータセットだけでいけます．このときの最適解は$\zeta^*(s, a)=\frac{d^\pi(s, a)}{d^{\mathcal{D}}(s, a)}$になります．

この手法では，次のDoubly robustな性質が成り立ちます：

$$
L\left(Q, \zeta^*\right)=L\left(Q^*, \zeta\right)=L\left(Q^*, \zeta^*\right)=\rho(\pi)
$$

つまり，$Q$か$\zeta$のどちらかが求まってればよいという意味で，Doubly robustです．

### Lagrangeを取る前に変形しよう

上で見たDoubly robustな手法は，１ステップのBellman誤差を使った評価を行いますが，これは一般にかなり不安定になります．
上の最適解は$Q$が$Q^\pi$に，$d$が$d^\pi$になりますが，どちらかが求まれば良いので，$d$についてだけ求めることを考えましょう．
そして，

$$
\hat{\rho}(\pi)=\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[\hat{\zeta}^*(s, a) \cdot R(s, a)\right]
$$

によって方策評価を行います．

まず，双対形式の方策評価を思い出しましょう：

$$
\begin{aligned}
\rho(\pi)=\max _{d \geq 0} & \sum_{s, a} d(s, a) \cdot R(s, a) \\
\text { s.t. } & d(s, a)=(1-\gamma) \mu_0(s) \pi(a \mid s)+\gamma \cdot \mathcal{P}_*^\pi d(s, a) \\
& \forall s \in S, a \in A
\end{aligned}
$$

これはOver-constrainedな問題であることに注意しましょう．
つまり，制約を満たすのは$d^\pi$だけであり，これは目的関数によらずに制限されます．
よって，これの目的を$\max_d -h(d)$に置き換えても，求まる$d$は結局$d^\pi$です．
これを使って，元の問題をそのまま解くのではなく，何かときやすい$h(d)$を使った別の問題をとくことを考えます．

---

**$h(d)=0$のとき**

このとき，Lagrangeを使ったオフライン方策評価は

$$
\begin{aligned}
\min _Q \max _\zeta & L(Q, \zeta) \\
= & (1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\
s 0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{\substack{\left(s, a, s^{\prime}\right) \sim d^{\mathcal{D}} \\
a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[\zeta(s, a) \cdot\left(\gamma Q\left(s^{\prime}, a^{\prime}\right)-Q(s, a)\right)\right]
\end{aligned}
$$

になります．一般に報酬がないほうが最適化しやすいことが知られているので（実験的にそうらしい？），$Q$と$\zeta$の両方を求めるよりも，こっちを特報が良いことがあります．
一方で，これは$Q$と$\zeta$の両方について線形なので，うまくやらないと無限大に吹っ飛んでしまいます．

---

**$h(d):=D_f\left(d \| d^{\mathcal{D}}\right)$のとき（Dual DICE）**

そこで，DualDICEでは次を解くことを考えます：

$$
\begin{aligned}
\max _d & -D_f\left(d \| d^{\mathcal{D}}\right) \\
\text { s.t. } & d(s, a)=(1-\gamma) \mu_0(s) \pi(a \mid s)+\gamma \cdot \mathcal{P}_*^\pi d(s, a) \\
& \forall s \in S, a \in A
\end{aligned}
$$

これを変形すると，

$$
\begin{aligned}
&\max _d \min _Q L(Q, d) \\
& :=-D_f\left(d \| d^{\mathcal{D}}\right)+\sum_{s, a} Q(s, a) \cdot\left((1-\gamma) \mu_0(s) \pi(a \mid s)+\gamma \cdot \mathcal{P}_*^\pi d(s, a)-d(s, a)\right)\\
& =(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]-D_f\left(d \| d^{\mathcal{D}}\right)+\sum_{s, a} Q(s, a) \cdot\left(\gamma \cdot \mathcal{P}_*^\pi d(s, a)-d(s, a)\right)
\end{aligned}
$$

です．
$\langle y, A x\rangle=\left\langle x, A_* y\right\rangle$を使えば，

$$
L(Q, d)=(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]-D_f\left(d \| d^{\mathcal{D}}\right)+\sum_{s, a} d(s, a) \cdot\left(\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)
$$
と等価です．
さらに$\zeta(s, a)=\frac{d(s, a)}{d^{\mathcal{D}}(s, a)}$を使えば，

$$
\begin{aligned}
& \max _\zeta \min _Q L(Q, \zeta) \\
:= & (1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]-\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}[f(\zeta(s, a))]+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[\zeta(s, a) \cdot\left(\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)\right]\\
& =(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{\left(s, a, s^{\prime}\right) \sim d^{\mathcal{D}}}\left[\zeta(s, a) \cdot\left(\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)-f(\zeta(s, a))\right]
\end{aligned}
$$

が成り立ちます．






---

**Fenchel-Rockafellar双対を使った解釈**

Dual DICEはFenchel-Rockafellar双対でも導出できます．
まず，上で見た双対問題は次の形式に直せます：

$$
\max _d-g(-A d)-h(d)
$$

ここで$g:=\delta_{\left\{(1-\gamma) \mu_0 \times \pi\right\}}$ and $A:=\gamma \cdot \mathcal{P}_*^\pi-I$
です．このFenchel-Rockafellar双対をとります．

* $A_*=\gamma \cdot \mathcal{P}^\pi-I$
* $h_*(\cdot)=\mathbb{E}_{d^{\mathcal{D}}}\left[f_*(\cdot)\right]$
* $g_*(\cdot)=(1-\gamma) \mathbb{E}_{\mu_0 \times \pi}[\cdot]$
  
に気をつければ，

$$
\begin{aligned}
& \min _Q g_*(Q)+h_*\left(A_* Q\right) \\
= & \min _Q(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[f_*\left(\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)\right] .
\end{aligned}
$$

が得られます．これを解くと，

$$
f_*^{\prime}\left(\gamma \cdot \mathcal{P}^\pi Q^*(s, a)-Q^*(s, a)\right)=\frac{d^\pi(s, a)}{d^{\mathcal{D}}(s, a)}
$$

によって$d^\pi$が得られます．例えば$f(x)=\frac{1}{2}x^2$とすれば，

$$
Q^*=\arg \min _Q(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\frac{1}{2} \mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[\left(\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)^2\right]
$$

であり，$\gamma \cdot \mathcal{P}^\pi Q^*(s, a)-Q^*(s, a)=\frac{d^\pi(s, a)}{d^{\mathcal{D}}(s, a)}$です．

## 方策の最適化

上では方策評価について確認しました．続いて方策の最適化をします．
まず，

$$
\rho(\pi)=\min _Q \max _{d \geq 0}(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) 
s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+ 
\sum_{s, a} d(s, a) \cdot\left(R(s, a)+\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right) .
$$

なので，この中身の関数を$L(Q, d; \pi)$をおきます．このとき，Danskinの定理から，

$$\frac{\partial}{\partial \pi} \rho(\pi)=\frac{\partial}{\partial \pi} \min _Q \max _{d \geq 0} L(Q, d ; \pi)=\frac{\partial}{\partial \pi} L\left(Q^*, d^* ; \pi\right)$$

が成り立ちます．ここで$Q^*$と$d^*$は$\min _Q \max _{d \geq 0} L(Q, d ; \pi)=\max _{d \geq 0} \min _Q L(Q, d ; \pi)$の解です．
この勾配を，$L$の項ごとに計算します．

$$\frac{\partial}{\partial \pi}(1-\gamma) \cdot \mathbb{E}_{\substack{a 0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q^*\left(s_0, a_0\right)\right]=(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q^*\left(s_0, a_0\right) \nabla \log \pi\left(a_0 \mid s_0\right)\right]$$

および

$$
\begin{aligned}
\frac{\partial}{\partial \pi} \mathbb{E}_{(s, a) \sim d^*}\left[R(s, a)+\gamma \cdot \mathcal{P}^\pi Q^*(s, a)-Q^*(s, a)\right]
&=\mathbb{E}_{(s, a) \sim d^*}\left[\gamma \cdot \frac{\partial}{\partial \pi} \mathbb{E}_{\substack{s^{\prime} \sim T(s, a) \\ a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[Q^*\left(s^{\prime}, a^{\prime}\right)\right]\right]\\
&=\gamma \cdot \mathbb{E}_{\substack{(s, a) \sim d^*, s^{\prime} \sim T(s, a) \\ a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[Q^*\left(s^{\prime}, a^{\prime}\right) \nabla \log \pi\left(a^{\prime} \mid s^{\prime}\right)\right]
\end{aligned}
$$

です．ここで，$Q^*=Q^\pi$，$d^*=d^\pi$，そして

$$
d^\pi(s, a)=(1-\gamma) \mu_0(s) \pi(a \mid s)+\gamma \pi(a \mid s) \sum_{\tilde{s}, \tilde{a}} T\left(s^{\prime} \mid \tilde{s}, \tilde{a}\right) d^\pi(\tilde{s}, \tilde{a})
$$

であることを使うと，結局

$$
\frac{\partial}{\partial \pi} L\left(Q^*, d^* ; \pi\right)=\mathbb{E}_{(s, a) \sim d^\pi}\left[Q^\pi(s, a) \nabla \log \pi(a \mid s)\right]
$$

が成り立ちます．これはよくある方策勾配法と同じですね！
このように，線型計画法を変形して方策勾配法を導出できます．

---

**オフライン方策勾配法**

これは簡単なので省略します．


---

**正則化が入ったときの勾配法（Primal AlgaeDICE）**

RLでは単純にReturnを最適化させる以外にも，何か正則化を入れたい場合があります．次の問題を考えてみましょう．

$$
\begin{aligned}
\rho(\pi)-D_f\left(d^\pi \| d^{\mathcal{D}}\right)=\max _d & -D_f\left(d \| d^{\mathcal{D}}\right)+\sum_{s, a} d(s, a) \cdot R(s, a) \\
\text { s.t. } d(s, a)=(1-\gamma) \mu_0(s) \pi(a \mid s)+\gamma \cdot \mathcal{P}_*^\pi d(s, a), & \forall s \in S, a \in A .
\end{aligned}
$$

Fenchel-Rockafellar双対を取ると，

$$
\max _\pi \min _Q(1-\gamma) \cdot \mathbb{E}_{\substack{a_0 \sim \pi\left(s_0\right) \\ s_0 \sim \mu_0}}\left[Q\left(s_0, a_0\right)\right]+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[f_*\left(R(s, a)+\gamma \cdot \mathcal{P}^\pi Q(s, a)-Q(s, a)\right)\right]
$$

を解けば，$\max _\pi \rho(\pi)-D_f\left(d^\pi \| d^{\mathcal{D}}\right)$が解けることがわかります．

これをやるのがPrimal AlgaeDICEアルゴリズムです．

備考：

* よく見ると，１つ目の項がOn-policyの方策最適化を，２つ目の項がOff-policyの方策最適化を行っていることがわかります．
* $f$-DivergenceとしてKLダイバージェンスを使うと，max-likelihood policy learningってやつに近い形が出てきます．
* 報酬を無視すれば模倣学習ができます

## Vを使った線型計画法

上で見た手法では，一旦$\pi$についての方策評価を挟んだ形で方策最適化を定義していましたが，こうすると二重の最適化問題になってしまいます．

次の線型計画法を使えば，一発で最適方策は求まります：

$$
\begin{aligned}
\max _{d \geq 0} & \sum_{s, a} d(s, a) \cdot R(s, a) \\
\text { s.t. } & \sum_a d(s, a)=(1-\gamma) \mu_0(s)+\gamma \cdot \mathcal{T}_* d(s), \\
& \forall s \in S,
\end{aligned}
$$

ここで$\mathcal{T}_* d(s):=\sum_{\tilde{s}, \tilde{a}} T(s \mid \tilde{s}, \tilde{a}) \cdot d(\tilde{s}, \tilde{a})$です．
また，

$$
\begin{aligned}
\min _V & (1-\gamma) \cdot \mathbb{E}_{s_0 \sim \mu_0}\left[V\left(s_0\right)\right] \\
\text { s.t. } & V(s) \geq R(s, a)+\gamma \cdot \mathcal{T} V(s, a), \\
& \forall s \in S, a \in A,
\end{aligned}
$$

もよく知られた形です．ここで，$\mathcal{T} V(s, a):=\mathbb{E}_{s^{\prime} \sim T(s, a)}\left[V\left(s^{\prime}\right)\right]$としました．

これをFenchel-Rockafellar双対を使って変形しましょう．特に$-D_f\left(d \| d^{\mathcal{D}}\right)$で正則化されたとき，次が得られます：

$V: S \rightarrow \mathbb{R}$ と $K: S \times A \rightarrow \mathbb{R}_{+}:$について，

$$
\min _{K \geq 0, V}(1-\gamma) \cdot \mathbb{E}_{s_0 \sim \mu_0}\left[V\left(s_0\right)\right]+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[f_*(K(s, a)+R(s, a)+\gamma \cdot \mathcal{T} V(s, a)-V(s))\right]
$$

一方で，これを解いても最適方策は得られません（最適$V^*$は得られますが）．そこで，次の事実を使います：

$$
d^*(s, a)=d^{\mathcal{D}}(s, a) \cdot f_*^{\prime}\left(K^*(s, a)+R(s, a)+\gamma \cdot \mathcal{T} V^*(s, a)-V^*(s)\right)
$$

よって，

$$
\pi^*(a \mid s)=\frac{d^*(s, a)}{\sum_{\tilde{a}} d^*(s, \tilde{a})}=\frac{d^{\mathcal{D}}(s, a) \cdot f_*^{\prime}\left(K^*(s, a)+R(s, a)+\gamma \cdot \mathcal{T} V^*(s, a)-V^*(s)\right)}{\sum_{\tilde{a}} d^{\mathcal{D}}(s, \tilde{a}) \cdot f_*^{\prime}\left(K^*(s, a)+R(s, \tilde{a})+\gamma \cdot \mathcal{T} V^*(s, \tilde{a})-V^*(s)\right)}
$$

が得られます．

備考：

* このf-DivergenceをKLに直すと，Max-likelihood policy learningってやつになります．
* よっと直すと方策評価にも使えます．

## 平均報酬の設定

次の平均報酬を考えましょう．

$$
\rho(\pi):=\lim _{t_{\text {stop }} \rightarrow \infty} \mathbb{E}\left[\frac{1}{t_{\text {stop }}} \sum_{t=0}^{t_{\text {stop }}} R\left(s_t, a_t\right) \mid s_0 \sim \mu_0, \forall t, a_t \sim \pi\left(s_t\right), s_{t+1} \sim T\left(s_t, a_t\right)\right]
$$

これは$\rho(\pi)=\mathbb{E}_{(s, a) \sim d^\pi}[R(s, a)]$のように期待値をとして表すことができ，ここで$d^\pi(s, a)=\mathcal{P}_*^\pi d^\pi(s, a) \quad \forall s \in S, a \in A$です．

これを使うと，次の線型計画法で平均報酬の方策評価ができます：

$$
\begin{aligned}
\rho(\pi)=\max _{d \geq 0} & \sum_{s, a} d(s, a) \cdot R(s, a) \\
\text { s.t. } & \sum_{s, a} d(s, a)=1 \text { and } \\
& d(s, a)=\mathcal{P}_*^\pi d(s, a), \forall s \in S, a \in A .
\end{aligned}
$$

上でやったように，これはOver-constrainedです．
よって目的関数を変えても$d^\pi$が求まります．
そこで，目的関数を$\max _d-D_f\left(d \| d^{\mathcal{D}}\right)$にして，

$$
\min _{Q, \lambda}-\lambda+\mathbb{E}_{(s, a) \sim d^{\mathcal{D}}}\left[f_*\left(\lambda+\mathcal{P}^\pi Q(s, a)-Q(s, a)\right)\right]
$$

を解くことを考えます．このとき

$$
f_*^{\prime}\left(\lambda^*+\mathcal{P}^\pi Q^*(s, a)-Q^*(s, a)\right)=\frac{d^\pi(s, a)}{d^{\mathcal{D}}(s, a)}
$$

によって$d^\pi$が得られます．上と同様にして，

$$
\max _\zeta \min _{Q, \lambda}-\lambda+\mathbb{E}_{\substack{\left(s, a, s^{\prime}\right) \sim d^{\mathcal{D}} \\ a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[\zeta(s, a) \cdot\left(\lambda+Q\left(s^{\prime}, a^{\prime}\right)-Q(s, a)\right)-f(\zeta(s, a))\right] .
$$

を解いても構いません．

---

**正則化を入れよう**

squareな正則化を入れると，

$$
\left.\max _\zeta \min _{Q, \lambda}-\lambda+\frac{1}{2} \lambda^2+\mathbb{E}_{\substack{\left(s, a, s^{\prime}\right) \sim d^{\mathcal{D}} \\ a^{\prime} \sim \pi\left(s^{\prime}\right)}}\left[\zeta(s, a) \cdot\left(\lambda+Q\left(s^{\prime}, a^{\prime}\right)-Q(s, a)+\frac{1}{4} Q^2(s, a)\right)\right)\right]
$$

が得られます．これはGenDICEで$\chi^2$-divergenceを使ったときの問題と同じです．



---

**方策の最適化**

省略